In [1]:
import koda.koda_fetch as kf
import koda.koda_parsing as kp

In [2]:
# Download GTFS data
operator = "xt"
date = "2023-01-05"
feed_type = "TripUpdates"
print(f"Fetching GTFS static data for {operator} on {date}")
static_zip_path = kf.fetch_gtfs_static(operator, date)
print(f"Fetching GTFS realtime data for {operator} on {date}")
rt_zip_path = kf.fetch_gtfs_realtime(operator, feed_type, date)

Fetching GTFS static data for xt on 2023-01-05
File already exists.
Fetching GTFS realtime data for xt on 2023-01-05
File already exists.


In [3]:
# Unzip and parse GTFS data
hour = "12"
minute = "11"
second = "00"

# NOTE: The static data is in GTFS txt files in a flat folder structure
static_folder_path = kp.unzip_data(static_zip_path)
print(f"Unzipped static data to {static_folder_path}")

# NOTE: The realtime data is in Protocol Buffer files in a nested folder structure
rt_folder_path = kp.unzip_data(rt_zip_path)
print(f"Unzipped realtime data to {rt_folder_path}")

# TODO: Figure out how to read cumulative data from multiple files without duplicates
df = kp.read_pb_to_dataframe(rt_folder_path, operator, feed_type, date[:4], date[5:7], date[8:], hour, minute, second)

print(f"Data for {operator} on {date} at {hour}:{minute}:{second}")
print(df.head())
print(df.iloc[0])

Unzipping ./dev_data/koda_download/xt-gtfs-static-2023-01-05.7z
File already unzipped.
Unzipped static data to ./dev_data/koda_data\xt-gtfs-static-2023-01-05
Unzipping ./dev_data/koda_download/xt-gtfs-realtime-2023-01-05.7z
File already unzipped.
Unzipped realtime data to ./dev_data/koda_data\xt-gtfs-realtime-2023-01-05
Data for xt on 2023-01-05 at 12:11:00
                   id tripUpdate_trip_tripId tripUpdate_trip_startDate  \
0  217990500819585594     217990000029717940                  20230105   
1  217990500824825347     217990000029118619                  20230105   
2  217990500824817629     217990000029112087                  20230105   
3  217990500822092765     217990000029733947                  20230105   
4  217990500816130965     217990000029699008                  20230105   

  tripUpdate_trip_scheduleRelationship  \
0                            SCHEDULED   
1                            SCHEDULED   
2                            SCHEDULED   
3                          